In [97]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [98]:
df=pd.read_csv('Advertising.csv')

In [99]:
x=df.drop('sales',axis=1)

In [100]:
x

,TV,radio,newspaper
0,230.1,37.8,69.2
1,44.5,39.3,45.1
2,17.2,45.9,69.3
3,151.5,41.3,58.5
4,180.8,10.8,58.4
...,...,...,...
195,38.2,3.7,13.8
196,94.2,4.9,8.1
197,177.0,9.3,6.4
198,283.6,42.0,66.2


In [101]:
y=df['sales']

In [102]:
y

0      22.1
1      10.4
2       9.3
3      18.5
4      12.9
       ... 
195     7.6
196     9.7
197    12.8
198    25.5
199    13.4
Name: sales, Length: 200, dtype: float64

In [103]:
from sklearn.preprocessing import PolynomialFeatures

In [104]:
polynomial_converter=PolynomialFeatures(degree=3,include_bias=False)

In [105]:
poly_features=polynomial_converter.fit_transform(x)

In [106]:
poly_features.shape

(200, 19)

In [107]:
from sklearn.model_selection import train_test_split

In [108]:
train_x,test_x,train_y,test_y=train_test_split(
    poly_features,y,test_size=0.3,random_state=101
)

In [109]:
train_x.shape

(140, 19)

# Scaling

In [110]:
from sklearn.preprocessing import StandardScaler

In [111]:
scaler=StandardScaler()

In [112]:
scaler.fit(train_x)

StandardScaler()

In [113]:
train_x=scaler.transform(train_x)

In [114]:
test_x=scaler.transform(test_x)

In [115]:
train_x[0]

array([ 0.49300171, -0.33994238,  1.61586707,  0.28407363, -0.02568776,
        1.49677566, -0.59023161,  0.41659155,  1.6137853 ,  0.08057172,
       -0.05392229,  1.01524393, -0.36986163,  0.52457967,  1.48737034,
       -0.66096022, -0.16360242,  0.54694754,  1.37075536])

In [116]:
poly_features[0]

array([2.30100000e+02, 3.78000000e+01, 6.92000000e+01, 5.29460100e+04,
       8.69778000e+03, 1.59229200e+04, 1.42884000e+03, 2.61576000e+03,
       4.78864000e+03, 1.21828769e+07, 2.00135918e+06, 3.66386389e+06,
       3.28776084e+05, 6.01886376e+05, 1.10186606e+06, 5.40101520e+04,
       9.88757280e+04, 1.81010592e+05, 3.31373888e+05])

# Ridge Regularization

In [117]:
from sklearn.linear_model import Ridge

In [118]:
help(Ridge)

Help on class Ridge in module sklearn.linear_model._ridge:

class Ridge(sklearn.base.MultiOutputMixin, sklearn.base.RegressorMixin, _BaseRidge)
 |  Ridge(alpha=1.0, *, fit_intercept=True, copy_X=True, max_iter=None, tol=0.0001, solver='auto', positive=False, random_state=None)
 |  
 |  Linear least squares with l2 regularization.
 |  
 |  Minimizes the objective function::
 |  
 |  ||y - Xw||^2_2 + alpha * ||w||^2_2
 |  
 |  This model solves a regression model where the loss function is
 |  the linear least squares function and regularization is given by
 |  the l2-norm. Also known as Ridge Regression or Tikhonov regularization.
 |  This estimator has built-in support for multi-variate regression
 |  (i.e., when y is a 2d-array of shape (n_samples, n_targets)).
 |  
 |  Read more in the :ref:`User Guide <ridge_regression>`.
 |  
 |  Parameters
 |  ----------
 |  alpha : {float, ndarray of shape (n_targets,)}, default=1.0
 |      Constant that multiplies the L2 term, controlling regula

In [119]:
ridge_model=Ridge(alpha=10)

In [120]:
ridge_model.fit(train_x,train_y)

Ridge(alpha=10)

In [121]:
test_predictions=ridge_model.predict(test_x)

In [122]:
from sklearn.metrics import mean_absolute_error,mean_squared_error,mean_absolute_percentage_error

In [123]:
mae=mean_absolute_error(test_y,test_predictions)

In [124]:
srmae=np.sqrt(mean_squared_error(test_y,test_predictions))

In [125]:
mae

0.5774404204714183

In [126]:
srmae

0.8946386461319685

# Use cross-validation to find best alpha value

In [127]:
from sklearn.linear_model import RidgeCV

In [128]:
help(RidgeCV)

Help on class RidgeCV in module sklearn.linear_model._ridge:

class RidgeCV(sklearn.base.MultiOutputMixin, sklearn.base.RegressorMixin, _BaseRidgeCV)
 |  RidgeCV(alphas=(0.1, 1.0, 10.0), *, fit_intercept=True, scoring=None, cv=None, gcv_mode=None, store_cv_values=False, alpha_per_target=False)
 |  
 |  Ridge regression with built-in cross-validation.
 |  
 |  See glossary entry for :term:`cross-validation estimator`.
 |  
 |  By default, it performs efficient Leave-One-Out Cross-Validation.
 |  
 |  Read more in the :ref:`User Guide <ridge_regression>`.
 |  
 |  Parameters
 |  ----------
 |  alphas : array-like of shape (n_alphas,), default=(0.1, 1.0, 10.0)
 |      Array of alpha values to try.
 |      Regularization strength; must be a positive float. Regularization
 |      improves the conditioning of the problem and reduces the variance of
 |      the estimates. Larger values specify stronger regularization.
 |      Alpha corresponds to ``1 / (2C)`` in other linear models such as
 |

In [129]:
ridge_cv_model=RidgeCV(alphas=(0.1,1,4,10),scoring='neg_mean_absolute_error')

In [130]:
ridge_cv_model.fit(train_x,train_y)

RidgeCV(alphas=(0.1, 1, 4, 10), scoring='neg_mean_absolute_error')

In [131]:
ridge_cv_model.alpha_

0.1

In [132]:
test_predictions=ridge_cv_model.predict(test_x)

In [133]:
mae=mean_absolute_error(test_y,test_predictions)

In [134]:
rmae=np.sqrt(mean_squared_error(test_y,test_predictions))

In [135]:
mae

0.42737748843373746

In [136]:
rmae

0.6180719926921404

In [137]:
ridge_cv_model.coef_

array([ 5.40769392,  0.5885865 ,  0.40390395, -6.18263924,  4.59607939,
       -1.18789654, -1.15200458,  0.57837796, -0.1261586 ,  2.5569777 ,
       -1.38900471,  0.86059434,  0.72219553, -0.26129256,  0.17870787,
        0.44353612, -0.21362436, -0.04622473, -0.06441449])

# Lasso Regularization

In [138]:
from sklearn.linear_model import LassoCV

In [139]:
lasso_cv_model=LassoCV(eps=0.001,n_alphas=100,cv=5,max_iter=1000000)

In [140]:
lasso_cv_model.fit(train_x,train_y)

LassoCV(cv=5, max_iter=1000000)

In [141]:
lasso_cv_model.alpha_

0.004943070909225827

In [142]:
test_predictions=lasso_cv_model.predict(test_x)

In [143]:
rmae=np.sqrt(mean_squared_error(test_y,test_predictions))

In [144]:
rmae

0.6063140748984036

In [145]:
mae=mean_absolute_error(test_y,test_predictions)

In [146]:
mae

0.43350346185900757

In [147]:
lasso_cv_model.coef_

array([ 4.86023329,  0.12544598,  0.20746872, -4.99250395,  4.38026519,
       -0.22977201, -0.        ,  0.07267717, -0.        ,  1.77780246,
       -0.69614918, -0.        ,  0.12044132, -0.        , -0.        ,
       -0.        ,  0.        ,  0.        , -0.        ])

# Elastic Regularization

In [148]:
from sklearn.linear_model import ElasticNetCV

In [150]:
elastic_model=ElasticNetCV(l1_ratio=[.1,.5,.7,.9,.95,.99,1],eps=0.001,n_alphas=100,max_iter=1000000)

In [151]:
elastic_model.fit(train_x,train_y)

ElasticNetCV(l1_ratio=[0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1], max_iter=1000000)

In [152]:
elastic_model.l1_ratio_

1.0

In [153]:
elastic_model.alpha_

0.004943070909225827

In [155]:
test_predictions=elastic_model.predict(test_x)

In [156]:
mae=mean_absolute_error(test_y,test_predictions)

In [157]:
rmae=np.sqrt(mean_squared_error(test_y,test_predictions))

In [159]:
mae

0.43350346185900757

In [160]:
rmae

0.6063140748984036